In [1]:
import os
os.chdir("../..")

In [2]:
import omegaconf
from shell.utils.experiment_utils import *
from shell.utils.metric import *
import matplotlib.pyplot as plt
from shell.fleet.network import TopologyGenerator
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd
from shell.fleet.fleet import Agent, Fleet
from shell.fleet.data.data_utilize import *
from shell.fleet.data.send import *

from sklearn.manifold import TSNE
from torchvision.utils import make_grid
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
seed_everything(0)

In [4]:
use_contrastive = True
num_tasks = 4
num_init_tasks = 2
dataset_name = "mnist"
bandwidth = 20

data_cfg = {
    "dataset_name": dataset_name,
    "num_tasks": num_tasks,
    "num_train_per_task": 128,
    "num_val_per_task": 102,
    'remap_labels': True,
    'use_contrastive': use_contrastive,
}
dataset = get_dataset(**data_cfg)

INFO:root:Class sequence: [5 0 3 4 2 6 7 1 8 9]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


In [5]:
seed_everything(7)
sender_dataset1 = get_dataset(**data_cfg)

INFO:root:Class sequence: [9 5 7 8 4 3 6 1 0 2]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


In [6]:
seed_everything(9)
sender_dataset2 = get_dataset(**data_cfg)

INFO:root:Class sequence: [9 1 6 5 2 0 4 8 3 7]
INFO:root:task 0 :(128, 1, 28, 28)
INFO:root:task 1 :(128, 1, 28, 28)
INFO:root:task 2 :(128, 1, 28, 28)
INFO:root:task 3 :(128, 1, 28, 28)


In [7]:
net_cfg = {
    'depth': num_init_tasks,
    'layer_size': 64,
    'num_init_tasks': num_init_tasks,
    'i_size': 28,
    'num_classes': 2,
    'num_tasks': 4,
    'dropout': 0.0,
}

agent_cfg = {
    'memory_size': 64,
    'use_contrastive': use_contrastive,
    'save_dir': 'test',
    'dataset_name': dataset_name,
}

In [8]:
NetCls = MLPSoftLLDynamic
LearnerCls = CompositionalDynamicER
AgentCls = SendDataAgent
sharing_cfg = DictConfig({
    "scorer": "cross_entropy",
    "num_queries": 5,
    'num_data_neighbors': 5,
    'num_filter_neighbors': 5,
    'num_coms_per_round': 2,
    "query_score_threshold": 0.0,
    "shared_memory_size": 50,
    "exploration_strategy": "epsilon_greedy",
    "min_epsilon": 0.1,
    "eps_decay_rate": 0.9,
    "init_epsilon": 0.8,
    "bandwidth": bandwidth,

})
train_cfg = {
    # "num_epochs": 40,
    "num_epochs": 1,
}


In [9]:
datasets = [dataset, sender_dataset1, sender_dataset2]
g = TopologyGenerator(num_nodes=len(datasets)).generate_fully_connected()
fleet = Fleet(g, 0, datasets,
              sharing_cfg, AgentCls, NetCls, LearnerCls, net_cfg, agent_cfg, 
              train_cfg)

INFO:root:Agent: node_id: 0, seed: 0
INFO:root:Agent: node_id: 1, seed: 1000
INFO:root:Agent: node_id: 2, seed: 2000
INFO:root:Created fleet with 3 agents
INFO:root:Adding neighbors...
INFO:root:Fleet initialized


In [10]:
receiver = fleet.agents[0]
sender = fleet.agents[1]
other = fleet.agents[2]

In [11]:
receiver.init_recommendation_engine()
sender.init_recommendation_engine()
other.init_recommendation_engine()

In [12]:
for t in range(num_init_tasks):
    fleet.train(t)

INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69484831	acc: 0.476
INFO:root:	task: avg	loss: 0.69484831	acc: 0.476
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69484831	acc: 0.476
INFO:root:	task: avg	loss: 0.69484831	acc: 0.476
INFO:root:final components: 2
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69867323	acc: 0.469
INFO:root:	task: avg	loss: 0.69867323	acc: 0.469
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.69867323	acc: 0.469
INFO:root:	task: avg	loss: 0.69867323	acc: 0.469
INFO:root:final components: 2
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.68871496	acc: 0.471
INFO:root:	task: avg	loss: 0.68871496	acc: 0.471
INFO:root:epochs: 0, training task: 0
INFO:root:	task: 0	loss: 0.68871496	acc: 0.471
INFO:root:	task: avg	loss: 0.68871496	acc: 0.471
INFO:root:final components: 2
INFO:root:epochs: 0, training task: 1
INFO:root:	task: 0	loss: 0.69484831	acc: 0.476
INFO:root:	task: 1	lo

In [13]:
sender.prepare_communicate(task_id=t, communication_round=0)

In [14]:
sender.communicate(task_id=t, communication_round=0)

In [15]:
receiver.incoming_data

{1: <torch.utils.data.dataset.Subset at 0x7fbb8e68e350>}